# Role of Metropolisation in Sampling Schemes
-----------

# Functions

## Packages

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.stats as stats
import scipy.special as special
import pandas as pd
import datetime


## Sampling schemes

In [0]:
def EulerMaruyama(q0, h, force, y, beta): 
    
    walkers = q0.shape[0]
    width = q0.shape[1]
    
    R_t = np.random.randn(walkers, width)
    
    q_t = q0 + h*force(q0,y) + np.sqrt(2*h/beta) * R_t

    return q_t, np.ones(walkers)

In [0]:
def MALA(q0, h, force, y, beta):

    q_t = q0
    walkers = q0.shape[0]
    width = q0.shape[1]
    
    R_t = np.random.randn(walkers, width)
    S_t = np.log(np.random.rand(walkers))
    q_tp1 = q_t + h*force(q_t, y) + np.sqrt(2*h/beta) * R_t

    q_tp1 = np.array(q_tp1)
    q_t = np.array(q_t)
    
    accept = S_t < np.minimum(np.zeros(walkers),
                             ((rho_log(q_tp1,y,beta) +
                             prop_dist_log(q_t,q_tp1,h,force,y, beta)) - 
                             (rho_log(q_t,y, beta) +
                             prop_dist_log(q_tp1,q_t,h,force,y,beta))))

    q_t[accept] = q_tp1[accept]

    return q_t, accept
  
def rho_log(q, y, beta):

    return -beta*U(q,y)
  
def prop_dist_log(x1, x2, h, force, y, beta):
    
    val = -(beta/(4*h))*(np.linalg.norm(x1-x2-h*force(x2,y),axis=1)**2)
    
    return val

In [0]:
def Heun(q0, h, force, y, beta):
    
    walkers = q0.shape[0]
    width = q0.shape[1]
    
    R_t = np.random.randn(walkers, width)
    
    q_hat = q0 + h*force(q0,y) + np.sqrt(2*h/beta) * R_t
    
    q_t = q0 + (h/2)*(force(q_hat, y) + force(q0,y)) + np.sqrt(2*h/beta) * R_t
    
    return q_t, np.ones(walkers)

In [0]:
def run_simulation( q0, Nsteps, h, step_function, force_function, y, beta=1):
    q_traj = [np.copy(q0)] 
    t_traj = [0]
    
    q = np.copy(q0) 
    t = 0 
    accepted = np.zeros(q0.shape[0])
    
    for n in range(Nsteps):
        q, acc = step_function(q, h, force_function, y, beta)
        t = t + h 

        q_traj += [q]   
        t_traj += [t] 
        
        accepted += acc
    
    q_traj = np.array(q_traj) 
    t_traj = np.array(t_traj)
    
    acceptance_rate = accepted / Nsteps

    return q_traj, t_traj, acceptance_rate

## Autocorrelation function

In [0]:
def acf(Phi,kmax):

    ac = np.zeros([kmax,Phi.shape[1]])
    mean_i = np.mean(Phi,axis=0)
    
    for k in range(kmax):
        Phi_shift = Phi[0:len(Phi)-k,:]
        ac[k] = np.mean((Phi[k:len(Phi),:] - mean_i)*
                        (Phi_shift - mean_i),axis=0)/np.mean((Phi - mean_i)**2,axis=0)
    
    return ac

In [0]:
def acf_1d_fast(Phi,kmax,acf_min = -1,burn_in = 0.1):

    Phi = Phi[int(burn_in*len(Phi)):len(Phi)]
    ac = np.zeros(kmax)
    mean_i = np.mean(Phi)
    for k in range(kmax):
        Phi_shift = Phi[0:len(Phi)-k]
        
        ac[k] = np.mean((Phi[k:len(Phi)] - mean_i) * 
                        (Phi_shift - mean_i))/np.mean((Phi-mean_i)**2)
        if ac[k] < acf_min:
            break
            
    return ac

# Sampling 1-D Case: Single Well

## Force function and potential energy

In [0]:
def force_Gaussian(q,y):

    F = -q 
    
    return F
    
def U(q,y):

    V = q**2 /2
    V = V[:,0]
    
    return V

In [0]:
# Generate a mixed Gaussian
N = 1000
a = 5+np.random.randn(int(N*0.2))
b = 8+np.random.randn(int(N*0.2))
c = 12+np.random.randn(int(N*0.4))
Y = np.concatenate((a, b, c), axis=0)

## Run simulation and plot the distribution

In [0]:
q0 = np.random.rand(100,1)*3

h_range = [2**x for x in range(-8,-4)]
Nsteps = 10000

beta =1

i=0

err_EM = np.zeros([len(h_range),q0.shape[1]])
err_MALA = np.zeros([len(h_range),q0.shape[1]])
for h in h_range:
    print(h)
    
    start_time = datetime.datetime.now()
    q_traj_EM, t_traj, acc_rate = run_simulation( q0 , Nsteps, h,  EulerMaruyama, force_Gaussian, Y, beta)
    end_time = datetime.datetime.now()
    print('EM done')
    print('time taken:'+str(end_time-start_time))
   
    start_time = datetime.datetime.now()
    q_traj_MALA, t_traj, acc_rate = run_simulation( q0 , Nsteps, h,  MALA, force_Gaussian, Y, beta)
    print('MALA done')
    print('mean acceptance rate of walkers = '+str(np.mean(acc_rate)))
    print('time taken:'+str(end_time-start_time))
    
    
    start_time = datetime.datetime.now()
    q_traj_Heun, t_traj, acc_rate = run_simulation( q0 , Nsteps, h,  Heun, force_Gaussian, Y, beta)
    end_time = datetime.datetime.now()
    print('Heun done')
    print('time taken:'+str(end_time-start_time))
    
    q_span = np.linspace(-2.5,2.5,100)
    Uq = (q_span**2)/2
    Z_beta = np.sqrt(np.pi*2/beta) # using wolfram alpha
    rho = np.exp(- 1 * Uq ) / Z_beta
    plt.plot( q_span, rho,':k',linewidth=3,label='Target rho_beta')
    
    histogram,bins = np.histogram(q_traj_EM,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='Euler-Maruyama')
    
    histogram,bins = np.histogram(q_traj_MALA,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='MALA')
    
    histogram,bins = np.histogram(q_traj_Heun,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='Heun')
    
    plt.title('h = '+str(h)+' starting unif(0,3)')
    plt.legend()
    plt.show()
    
    i+=1

## ACF plot

In [0]:
kmax = int(q_traj_EM.shape[0]*0.3)

for i in range(q_traj_EM.shape[2]):
    plt.plot(np.arange(kmax),np.mean(acf(q_traj_EM[:,:,i],kmax),axis=1),label='EM - $\mu$'+str(i))
for i in range(q_traj_MALA.shape[2]):
    plt.plot(np.arange(kmax),np.mean(acf(q_traj_MALA[:,:,i],kmax),axis=1),label='MALA - $\mu$'+str(i))
for i in range(q_traj_MALA.shape[2]):
    plt.plot(np.arange(kmax),np.mean(acf(q_traj_Heun[:,:,i],kmax),axis=1),label='Heun - $\mu$'+str(i))

plt.legend()
plt.title('Single Well - starting unif(0,3) - Mean ACF for $\mu$ over '+str(q_traj_EM.shape[1])+' walkers')
plt.xlabel('k shift')
plt.ylabel('acf')
plt.show()

## Optimal acceptance rate and step size (MALA)

In [0]:
q0 = np.random.rand(30,1)
h = 0.0001 

h_range = np.concatenate((np.arange(0.001,1,0.02),np.arange(0.9,1,0.01),np.arange(1,10,0.2)))

beta = 0.1
tau = []
acceptance = []
Nsteps = 500

for h in h_range:
    print('h='+str(h))

    start_time = datetime.datetime.now()
    q_traj, t_traj,acc_rate = run_simulation( q0 , Nsteps, h,  MALA, force_Gaussian, Y,beta)
    end_time = datetime.datetime.now()
    
    q_traj_MALA = q_traj
    acceptance += [np.mean(acc_rate)]
    
    print('MALA_done, h='+str(h))
    print('acceptance: '+str(acceptance[-1]))
    print('time taken: '+str(end_time-start_time))

    
    histogram,bins = np.histogram(q_traj_MALA,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='MALA - h='+str(h))
    
    Phi = np.mean(q_traj,axis=1)

    start_time = datetime.datetime.now()
    ac = acf_1d_fast(Phi,int(Nsteps*0.8),0.01,0.1)
    tau_Phi = 1 + 2*sum(ac)
    end_time = datetime.datetime.now()
    
    print('tau='+str(tau_Phi))
    print('time taken:'+str(end_time-start_time))
    
    tau += [tau_Phi]
    
    print('compare var ='+str(np.var(np.mean(q_traj,axis=0))))
    print('with '+str(tau_Phi/Nsteps))
    print('\n')

In [0]:
fig = plt.figure(figsize=[8,6])
plt.semilogy()
plt.scatter(acceptance,np.array(tau)/Nsteps)
plt.xlabel('acceptance')
plt.ylabel('tau /N')
plt.title('Optimal Acceptance Rate - Single well, beta='+str(beta)+' - MALA, '+str(Nsteps)+' steps')
plt.show()

In [0]:
fig = plt.figure(figsize=[8,6])
plt.semilogy()
plt.scatter(h_range,np.array(tau)/Nsteps)
plt.xlabel('h')
plt.ylabel('tau /N')
plt.title('Optimal Step Size - Single well, beta='+str(beta)+' - MALA, '+str(Nsteps)+' steps')
plt.show()

## KL divergence and L2 error

In [0]:
q0 = np.zeros([1000,1])
h_range = np.linspace(0.3,0.8, 100)
T = 2*10**1

KL_EM = np.zeros(len(h_range))
KL_Heun = np.zeros(len(h_range))
KL_MALA = np.zeros(len(h_range))
L2_EM = np.zeros(len(h_range))
L2_Heun = np.zeros(len(h_range))
L2_MALA = np.zeros(len(h_range))

beta = 1

Z_beta = 2.50662

i=0
err_EM = np.zeros([len(h_range),q0.shape[1]])
err_MALA = np.zeros([len(h_range),q0.shape[1]])
err_Heun = np.zeros([len(h_range),q0.shape[1]])

for h in h_range:
    print(h)
    Nsteps = 1000
    
    start_time = datetime.datetime.now()
    q_traj_EM, t_traj, acc = run_simulation( q0 , Nsteps, h,  EulerMaruyama, force_Gaussian, Y, beta)
    end_time = datetime.datetime.now()
    print('EM done')
    print('time taken:'+str(end_time-start_time))
    
    start_time = datetime.datetime.now()
    q_traj_MALA, t_traj, acc = run_simulation( q0 , Nsteps, h,  MALA, force_Gaussian, Y, beta)
    end_time = datetime.datetime.now()
    print('MALA done')
    print('time taken:'+str(end_time-start_time))
    
    start_time = datetime.datetime.now()
    q_traj_Heun, t_traj, acc = run_simulation( q0 , Nsteps, h,  Heun, force_Gaussian, Y, beta)
    end_time = datetime.datetime.now()
    print('Heun done')
    print('time taken:'+str(end_time-start_time))

    
    histogram,bins = np.histogram(q_traj_EM,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    a = (midx**2)/2
    rho = np.exp(- beta * a ) / Z_beta
    L2_EM[i] = np.linalg.norm(histogram-rho)
    KL_EM[i] = stats.entropy(rho, histogram)    
    
    q_span = np.linspace(-2.5, 2.5,100)
    Uq = (q_span**2)/2
    rho = np.exp(- beta * Uq ) / Z_beta
    
    histogram,bins = np.histogram(q_traj_MALA,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    a = (midx**2)/2
    rho = np.exp(- beta * a ) / Z_beta
    L2_MALA[i] = np.linalg.norm(histogram-rho)
    KL_MALA[i] = stats.entropy(rho, histogram)
    
    histogram,bins = np.histogram(q_traj_Heun,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    a = (midx**2)/2
    rho = np.exp(- beta * a ) / Z_beta
    L2_Heun[i] = np.linalg.norm(histogram-rho)
    KL_Heun[i] = stats.entropy(rho, histogram)
    
    i+=1

In [0]:
# Plotting KL divergence
fig = plt.figure(figsize = [8,6])
plt.semilogy(h_range, pow(np.array(h_range), 1),'b--', label = '$O(h^2)$')
plt.semilogy(h_range, pow(np.array(h_range), 4),'g--', label = '$O(h^4)$')
plt.scatter(h_range, KL_EM, label = 'EM')
plt.scatter(h_range, KL_MALA, label = 'MALA')
plt.scatter(h_range, KL_Heun, label = 'Heun')
plt.title('$L_2$ Error- Double Well \n beta={} \n Nsteps = {}'.format(beta, Nsteps))
plt.xlabel('Stepsize')
plt.ylabel('$L_2$ Error')
plt.legend()
plt.show()


# Plotting L_2 Error
fig = plt.figure(figsize = [8,6])
plt.semilogy(h_range, pow(np.array(h_range), 1),'b--', label = '$O(h)$')
plt.semilogy(h_range, pow(np.array(h_range), 2),'g--', label = '$O(h^2)$')
plt.scatter(h_range, L2_EM, label = 'EM')
plt.scatter(h_range, L2_MALA, label = 'MALA')
plt.scatter(h_range, L2_Heun, label = 'Heun')
plt.title('$L_2$ Error- Double Well \n beta={} \n Nsteps = {}'.format(beta, Nsteps))
plt.xlabel('Stepsize')
plt.ylabel('$L_2$ Error')
plt.legend()
plt.show()

# Sampling 1-D Case: Double Well

## Force function and potential energy

In [0]:
def DoubleWellForce(q,y):

    F = -8*q*(q**2-1)

    return F

def U(q,y):

    V = 2*(q**2 - 1)**2
    V = V[:,0]
    
    return V

## Run simulation and plot the distribution

In [0]:
q0 = np.random.rand(100,1) *3
h_range = [2**x for x in range(-8,-4)]
Nsteps = 10000

beta = 1

i=0
err_EM = np.zeros([len(h_range),q0.shape[1]])
err_MALA = np.zeros([len(h_range),q0.shape[1]])
for h in h_range:
    print(h)
    start_time = datetime.datetime.now()
    q_traj_EM, t_traj, acc_rate = run_simulation( q0 , Nsteps, h,  EulerMaruyama, DoubleWellForce, Y, beta)
    end_time = datetime.datetime.now()
    print('EM done')
    print('time taken:'+str(end_time-start_time))
    
    start_time = datetime.datetime.now()
    q_traj_MALA, t_traj, acc_rate = run_simulation( q0 , Nsteps, h,  MALA, DoubleWellForce, Y, beta)
    end_time = datetime.datetime.now()
    print('MALA done')
    print('mean acceptance rate of walkers = '+str(np.mean(acc_rate)))
    print('time taken:'+str(end_time-start_time))

    start_time = datetime.datetime.now()
    q_traj_Heun, t_traj, acc_rate = run_simulation( q0 , Nsteps, h,  Heun, DoubleWellForce, Y, beta)
    end_time = datetime.datetime.now()
    print('Heun done')
    print('time taken:'+str(end_time-start_time))
    
    
    q_span = np.linspace(-2.5,2.5,100)
    Uq = 2*(q_span**2-1)**2
    Z_beta = 0.5*np.pi*np.exp(-beta)*(special.iv(-0.25,beta) + special.iv(0.25,beta)) # using wolfram alpha
    rho = np.exp(- 1 * Uq ) / Z_beta
    plt.plot( q_span, rho,':k',linewidth=3,label='Target rho_beta')
    
    histogram,bins = np.histogram(q_traj_EM,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='Euler-Maruyama')
    
    histogram,bins = np.histogram(q_traj_MALA,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='MALA')
    plt.title('h = '+str(h)+' starting at unif(0,3)')
    plt.legend()
    plt.show()
    
    i+=1

## ACF plot

In [0]:
kmax = int(q_traj_EM.shape[0]*0.3)

for i in range(q_traj_EM.shape[2]):
    plt.plot(np.arange(kmax),np.mean(acf(q_traj_EM[:,:,i],kmax),axis=1),label='EM - $\mu$'+str(i))
for i in range(q_traj_MALA.shape[2]):
    plt.plot(np.arange(kmax),np.mean(acf(q_traj_MALA[:,:,i],kmax),axis=1),label='MALA - $\mu$'+str(i))
for i in range(q_traj_MALA.shape[2]):
    plt.plot(np.arange(kmax),np.mean(acf(q_traj_Heun[:,:,i],kmax),axis=1),label='Heun - $\mu$'+str(i))

plt.legend()
plt.title('Double Well - starting unif(0,3) - Mean ACF for $\mu$ over '+str(q_traj_EM.shape[1])+' walkers')
plt.xlabel('k shift')
plt.ylabel('acf')
plt.show()

## Optimal acceptance rate and step size ( MALA)

In [0]:
q0 = np.random.rand(30,1)
h = 0.0001 

h_range = np.concatenate((np.arange(0.001,1,0.02),np.arange(0.9,1,0.01),np.arange(1,10,0.2)))

beta = 0.1
tau = []
acceptance = []
Nsteps = 500

for h in h_range:
    print('h='+str(h))

    start_time = datetime.datetime.now()
    q_traj, t_traj,acc_rate = run_simulation( q0 , Nsteps, h,  MALA, DoubleWellForce, Y,beta)
    end_time = datetime.datetime.now()
    
    q_traj_MALA = q_traj
    acceptance += [np.mean(acc_rate)]
    
    print('MALA_done, h='+str(h))
    print('acceptance: '+str(acceptance[-1]))
    print('time taken: '+str(end_time-start_time))

    
    histogram,bins = np.histogram(q_traj_MALA,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='MALA - h='+str(h))
    
    Phi = np.mean(q_traj,axis=1)

    start_time = datetime.datetime.now()
    ac = acf_1d_fast(Phi,int(Nsteps*0.8),0.01,0.1)
    tau_Phi = 1 + 2*sum(ac)
    end_time = datetime.datetime.now()
    
    print('tau='+str(tau_Phi))
    print('time taken:'+str(end_time-start_time))
    
    tau += [tau_Phi]
    
    print('compare var ='+str(np.var(np.mean(q_traj,axis=0))))
    print('with '+str(tau_Phi/Nsteps))
    print('\n')

In [0]:
fig = plt.figure(figsize=[8,6])
plt.semilogy()
plt.scatter(acceptance,np.array(tau)/Nsteps)
plt.xlabel('acceptance')
plt.ylabel('tau /N')
plt.title('Optimal Acceptance Rate - Double well, beta='+str(beta)+' - MALA, '+str(Nsteps)+' steps')
plt.show()

In [0]:
fig = plt.figure(figsize=[8,6])
plt.semilogy()
plt.scatter(h_range,np.array(tau)/Nsteps)
plt.xlabel('h')
plt.ylabel('tau /N')
plt.title('Optimal Step Size - Double well, beta='+str(beta)+' - MALA, '+str(Nsteps)+' steps')
plt.show()

## KL convergence and L2 error

In [0]:
q0 = np.zeros([3000,1])
h_range = np.linspace(0.001,0.04, 100)
T = 2*10**1

KL_EM = np.zeros(len(h_range))
KL_Heun = np.zeros(len(h_range))
KL_MALA = np.zeros(len(h_range))

L2_EM = np.zeros(len(h_range))
L2_Heun = np.zeros(len(h_range))
L2_MALA = np.zeros(len(h_range))

beta = 1

Z_beta = 0.5*np.pi*np.exp(-beta)*(special.iv(-0.25,beta) + special.iv(0.25,beta)) # using wolfram alpha


i=0
err_EM = np.zeros([len(h_range),q0.shape[1]])
err_MALA = np.zeros([len(h_range),q0.shape[1]])
err_Heun = np.zeros([len(h_range),q0.shape[1]])

for h in h_range:
    print(h)
    Nsteps = 1000
    
    start_time = datetime.datetime.now()
    q_traj_EM, t_traj, acc = run_simulation( q0 , Nsteps, h,  EulerMaruyama, DoubleWellForce, Y, beta)
    end_time = datetime.datetime.now()
    print('EM done')
    print('time taken:'+str(end_time-start_time))
    
    start_time = datetime.datetime.now()
    q_traj_MALA, t_traj, acc= run_simulation( q0 , Nsteps, h,  MALA, DoubleWellForce, Y, beta)
    end_time = datetime.datetime.now()
    print('MALA done')
    print('time taken:'+str(end_time-start_time))
    
    start_time = datetime.datetime.now()
    q_traj_Heun, t_traj, acc = run_simulation( q0 , Nsteps, h,  Heun, DoubleWellForce, Y, beta)
    end_time = datetime.datetime.now()
    print('Heun done')
    print('time taken:'+str(end_time-start_time))

    
    histogram,bins = np.histogram(q_traj_EM,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    a = 2*(midx**2-1)**2
    rho = np.exp(- beta * a ) / Z_beta
    L2_EM[i] = np.linalg.norm(histogram-rho)
    KL_EM[i] = stats.entropy(rho, histogram)

    
    q_span = np.linspace(-2.5, 2.5,100)
    Uq = 2*(q_span**2-1)**2
    rho = np.exp(- beta * Uq ) / Z_beta
    
    histogram,bins = np.histogram(q_traj_MALA,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    a = 2*(midx**2-1)**2
    rho = np.exp(- beta * a ) / Z_beta
    L2_MALA[i] = np.linalg.norm(histogram-rho)
    KL_MALA[i] = stats.entropy(rho, histogram)
    
    histogram,bins = np.histogram(q_traj_Heun,bins=50,density=True)
    midx = (bins[0:-1]+bins[1:])/2
    a = 2*(midx**2-1)**2
    rho = np.exp(- beta * a ) / Z_beta
    L2_Heun[i] = np.linalg.norm(histogram-rho)
    KL_Heun[i] = stats.entropy(rho, histogram)
    
    i+=1

In [0]:
# Plotting KL divergence
fig = plt.figure(figsize = [8,6])
plt.semilogy(h_range, pow(np.array(h_range), 1),'b--', label = '$O(h^2)$')
plt.semilogy(h_range, pow(np.array(h_range), 4),'g--', label = '$O(h^4)$')
plt.scatter(h_range, KL_EM, label = 'EM')
plt.scatter(h_range, KL_MALA, label = 'MALA')
plt.scatter(h_range, KL_Heun, label = 'Heun')
plt.title('$L_2$ Error- Double Well \n beta={} \n Nsteps = {}'.format(beta, Nsteps))
plt.xlabel('Stepsize')
plt.ylabel('$L_2$ Error')
plt.legend()
plt.show()


# Plotting L_2 Error
fig = plt.figure(figsize = [8,6])
plt.semilogy(h_range, pow(np.array(h_range), 1),'b--', label = '$O(h)$')
plt.semilogy(h_range, pow(np.array(h_range), 2),'g--', label = '$O(h^2)$')
plt.scatter(h_range, L2_EM, label = 'EM')
plt.scatter(h_range, L2_MALA, label = 'MALA')
plt.scatter(h_range, L2_Heun, label = 'Heun')
plt.title('$L_2$ Error- Double Well \n beta={} \n Nsteps = {}'.format(beta, Nsteps))
plt.xlabel('Stepsize')
plt.ylabel('$L_2$ Error')
plt.legend()
plt.show()

# Bayesian Inference

## Force function and potential energy

In [0]:
def U(q,y):

    walkers = q.shape[0]
    width = q.shape[1]
    
    V = np.zeros(walkers)
    m = np.mean(y)
    s2 = np.var(y)**2
    
    for k in range(walkers):
        rho_prior = np.prod([np.exp(-(m - q[k,j])**2 / s2) for j in range(width)])
        
        V[k] = -np.log(rho_prior) - sum([np.log(sum([np.exp(-(y[i]-q[k,j])**2/2) 
                                                     for j in range(width)])) 
                                         for i in range(len(y))])
    
    return V
    
def force(q, y):

    walkers = q.shape[0]
    width = q.shape[1]
    
    F = np.zeros(q.shape)
    s2 = np.var(y)**2
    m = np.mean(y)

    for i in range(walkers):
        denom = np.sum([(np.exp(-((y-q[i,J])**2)/2)) for J in range(width)])
        for j in range(width):
            F[i, j] = (m-q[i, j])/s2
            + np.sum([(np.exp(-((y-q[i,j])**2)/2)*(y-q[i,j]))/denom])


    return(F)

## Tomato data set

### Load data and plot the histogram

In [0]:
Y=np.load('tomatoes.npy')
fig = plt.figure(figsize=[8,6])
plt.hist(Y,bins=10, density = True, edgecolor = 'k')
plt.title('Tomato Data')
plt.xlabel('Mass (g)')
plt.ylabel('Density')
plt.show()

### Run simulation

In [0]:
q0 = np.random.rand(20,3)*10+10 
h = 0.0001 
Nsteps = 5000
beta = 1

start_time = datetime.datetime.now()
q_traj_EM, t_traj_EM, acc = run_simulation( q0 , Nsteps, h,  EulerMaruyama, force, Y, beta)
end_time = datetime.datetime.now()
print('time taken:'+str(end_time-start_time))

start_time = datetime.datetime.now()
q_traj_MALA, t_traj_MALA, acc = run_simulation( q0 , Nsteps, h, MALA, force, Y, beta)
end_time = datetime.datetime.now()
print('time taken:'+str(end_time-start_time))

start_time = datetime.datetime.now()
q_traj_Heun, t_traj_Heun, acc = run_simulation( q0 , Nsteps, h,  Heun, force, Y, beta)
end_time = datetime.datetime.now()
print('time taken:'+str(end_time-start_time))


### Unsorted plot

In [0]:
fig = plt.figure(figsize=[15,4])
max_q = np.amax(q_traj_EM)
min_q = max(np.amin(q_traj_EM),0) # limits minimum to zero
for i in range(q_traj_EM.shape[2]):
    qn = q_traj_EM[:,:,i]
    histogram,bins = np.histogram(qn,bins=50,range=[min_q,max_q], density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='$\mu$'+str(i))
plt.title('Distribution of $\mu$ - EM')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.hist(Y,bins=10,density=True)
plt.legend()
plt.show()

fig = plt.figure(figsize=[15,4])
max_q = np.amax(q_traj_MALA)
min_q = max(np.amin(q_traj_MALA),0) # limits minimum to zero
for i in range(q_traj_MALA.shape[2]):
    qn = q_traj_MALA[:,:,i]
    histogram,bins = np.histogram(qn,bins=50,range=[min_q,max_q], density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='$\mu$'+str(i))
plt.title('Distribution of $\mu$ - MALA')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.hist(Y,bins=10,density=True)
plt.legend()
plt.show()

fig = plt.figure(figsize=[15,4])
max_q = np.amax(q_traj_Heun)
min_q = max(np.amin(q_traj_Heun),0) # limits minimum to zero
for i in range(q_traj_Heun.shape[2]):
    qn = q_traj_Heun[:,:,i]
    histogram,bins = np.histogram(qn,bins=50,range=[min_q,max_q], density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='$\mu$'+str(i))
plt.title('Distribution of $\mu$ - Heun')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.hist(Y,bins=10,density=True)
plt.legend()
plt.show()

### Sorted plot

In [0]:
q_traj_EM_sort = np.sort(q_traj_EM, axis=2)
fig = plt.figure(figsize=[15,4])
max_q = np.amax(q_traj_EM)
min_q = max(np.amin(q_traj_EM),0) # limits minimum to zero
for i in range(q_traj_EM.shape[2]):
    qn = q_traj_EM_sort[:,:,i]
    histogram,bins = np.histogram(qn,bins=50,range=[min_q,max_q], density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='$\mu$'+str(i))
plt.title('Distribution of sorted $\mu$ - EM')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.hist(Y,bins=10,density=True)
plt.legend()
plt.show()

q_traj_MALA_sort = np.sort(q_traj_MALA, axis=2)
fig = plt.figure(figsize=[15,4])
max_q = np.amax(q_traj_MALA)
min_q = max(np.amin(q_traj_MALA),0) # limits minimum to zero
for i in range(q_traj_MALA.shape[2]):
    qn = q_traj_MALA_sort[:,:,i]
    histogram,bins = np.histogram(qn,bins=50,range=[min_q,max_q], density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='$\mu$'+str(i))
plt.title('Distribution of sorted $\mu$ - MALA')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.hist(Y,bins=10,density=True)
plt.legend()
plt.show()

q_traj_Heun_sort = np.sort(q_traj_Heun, axis=2)
fig = plt.figure(figsize=[15,4])
max_q = np.amax(q_traj_Heun)
min_q = max(np.amin(q_traj_Heun),0) # limits minimum to zero
for i in range(q_traj_Heun.shape[2]):
    qn = q_traj_Heun_sort[:,:,i]
    histogram,bins = np.histogram(qn,bins=50,range=[min_q,max_q], density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='$\mu$'+str(i))
plt.title('Distribution of sorted $\mu$ - Heun')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.hist(Y,bins=10,density=True)
plt.legend()
plt.show()

## Stamps data set

### Load data and plot the histogram

In [0]:
def readData(DataFile):
    datapoints = []
    a = 1
    openfile = open(DataFile, 'r')
    for line in openfile:
        a1 = line.split()
        datapoints.extend(a1)
    openfile.close()
    return(datapoints)
file = readData('Stamp.dat')
Y = pd.to_numeric(list(file[1:486]))
Groupings = pd.to_numeric(list(file[487:len(file)-1]))
Y = Y*100
fig = plt.figure(figsize=[8,6])
plt.hist(Y,bins=50, density = True, edgecolor = 'k')
plt.title('Hidalgo Stamp Data')
plt.xlabel('Thickness (100nm)')
plt.ylabel('Density')
plt.show()

### Run simulation

In [0]:
q0 = np.random.rand(20,3)*10+10 
h = 0.0001 
Nsteps = 5000
beta = 1

start_time = datetime.datetime.now()
q_traj_EM, t_traj_EM, acc = run_simulation( q0 , Nsteps, h,  EulerMaruyama, force, Y, beta)
end_time = datetime.datetime.now()
print('time taken:'+str(end_time-start_time))

start_time = datetime.datetime.now()
q_traj_MALA, t_traj_MALA, acc = run_simulation( q0 , Nsteps, h, MALA, force, Y, beta)
end_time = datetime.datetime.now()
print('time taken:'+str(end_time-start_time))

start_time = datetime.datetime.now()
q_traj_Heun, t_traj_Heun, acc = run_simulation( q0 , Nsteps, h,  Heun, force, Y, beta)
end_time = datetime.datetime.now()
print('time taken:'+str(end_time-start_time))

### Unsorted plot

In [0]:
fig = plt.figure(figsize=[15,4])
max_q = np.amax(q_traj_EM)
min_q = max(np.amin(q_traj_EM),0) # limits minimum to zero
for i in range(q_traj_EM.shape[2]):
    qn = q_traj_EM[:,:,i]
    histogram,bins = np.histogram(qn,bins=50,range=[min_q,max_q], density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='$\mu$'+str(i))
plt.title('Distribution of $\mu$ - EM')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.hist(Y,bins=10,density=True)
plt.legend()
plt.show()

fig = plt.figure(figsize=[15,4])
max_q = np.amax(q_traj_MALA)
min_q = max(np.amin(q_traj_MALA),0) # limits minimum to zero
for i in range(q_traj_MALA.shape[2]):
    qn = q_traj_MALA[:,:,i]
    histogram,bins = np.histogram(qn,bins=50,range=[min_q,max_q], density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='$\mu$'+str(i))
plt.title('Distribution of $\mu$ - MALA')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.hist(Y,bins=10,density=True)
plt.legend()
plt.show()

fig = plt.figure(figsize=[15,4])
max_q = np.amax(q_traj_Heun)
min_q = max(np.amin(q_traj_Heun),0) # limits minimum to zero
for i in range(q_traj_Heun.shape[2]):
    qn = q_traj_Heun[:,:,i]
    histogram,bins = np.histogram(qn,bins=50,range=[min_q,max_q], density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='$\mu$'+str(i))
plt.title('Distribution of $\mu$ - Heun')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.hist(Y,bins=10,density=True)
plt.legend()
plt.show()

### Sorted plot

In [0]:
q_traj_EM_sort = np.sort(q_traj_EM, axis=2)
fig = plt.figure(figsize=[15,4])
max_q = np.amax(q_traj_EM)
min_q = max(np.amin(q_traj_EM),0) # limits minimum to zero
for i in range(q_traj_EM.shape[2]):
    qn = q_traj_EM_sort[:,:,i]
    histogram,bins = np.histogram(qn,bins=50,range=[min_q,max_q], density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='$\mu$'+str(i))
plt.title('Distribution of sorted $\mu$ - EM')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.hist(Y,bins=10,density=True)
plt.legend()
plt.show()

q_traj_MALA_sort = np.sort(q_traj_MALA, axis=2)
fig = plt.figure(figsize=[15,4])
max_q = np.amax(q_traj_MALA)
min_q = max(np.amin(q_traj_MALA),0) # limits minimum to zero
for i in range(q_traj_MALA.shape[2]):
    qn = q_traj_MALA_sort[:,:,i]
    histogram,bins = np.histogram(qn,bins=50,range=[min_q,max_q], density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='$\mu$'+str(i))
plt.title('Distribution of sorted $\mu$ - MALA')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.hist(Y,bins=10,density=True)
plt.legend()
plt.show()

q_traj_Heun_sort = np.sort(q_traj_Heun, axis=2)
fig = plt.figure(figsize=[15,4])
max_q = np.amax(q_traj_Heun)
min_q = max(np.amin(q_traj_Heun),0) # limits minimum to zero
for i in range(q_traj_Heun.shape[2]):
    qn = q_traj_Heun_sort[:,:,i]
    histogram,bins = np.histogram(qn,bins=50,range=[min_q,max_q], density=True)
    midx = (bins[0:-1]+bins[1:])/2
    plt.plot(midx,histogram,label='$\mu$'+str(i))
plt.title('Distribution of sorted $\mu$ - Heun')
plt.xlabel('$q$')
plt.ylabel('Density')
plt.hist(Y,bins=10,density=True)
plt.legend()
plt.show()